# <span style='color:#599084'> Telecom Churn Case Study(Machine learning -II) - Siva Pothuru & Vigneshwari Chandramohan

## <span style='color:#0099CC'>Problem Statement

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

## <span style='color:#0099CC'>Business Goal

Identify customers at high risk of churn and identify the main indicators of churn

## <span style='color:#0099CC'>Solution

Build predictive models to identify customers at high risk of churn and identify the main indicators of churn

In [ ]:
# Importing necessary Library
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn import metrics


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_roc_curve

from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn import model_selection


***Reading the Telecom Churn data file***

In [ ]:
telecom_df = pd.read_csv('telecom_churn_data.csv')
pd.options.display.max_columns = 999
telecom_df.head(10)

In [ ]:
telecom_df.shape

In [ ]:
telecom_df.info(verbose = True)

## <span style='color:#0099CC'>Checking for Null Values

In [ ]:
pd.options.display.max_rows = 999
(telecom_df.isnull().sum()/telecom_df.shape[0])*100

***Checking if there are meaningful missing and Actual missing rows***

**Meaningful Missing** - If the `date_of_last_rech_6`, `max_rech_amt_6`, `total_rech_amt_6` has 'NA' values, impute those values with '0' <br>
**Actual Missing** - If the `date_of_last_rech_6`, `max_rech_amt_6` has proper values and  `total_rech_amt_6` has 'NA' values, impute the 'NA' values in `total_rech_amt_6` with mean, median or mode.

In [ ]:
telecom_df[telecom_df['total_rech_num_6'] == 0][['total_rech_amt_6','max_rech_amt_6','monthly_2g_6','monthly_3g_6','sachet_2g_6','sachet_3g_6']].nunique()

In [ ]:
telecom_df[(telecom_df['total_rech_amt_6'] != 0) & (telecom_df['max_rech_amt_6'] != 0) & (telecom_df['total_rech_num_6'] == 0)][['total_rech_num_6','date_of_last_rech_6']]



In [ ]:
telecom_df[telecom_df['total_rech_num_7'] == 0][['total_rech_amt_7','max_rech_amt_7','monthly_2g_7','monthly_3g_7','sachet_2g_7','sachet_3g_7']].nunique()

In [ ]:
telecom_df[telecom_df['total_rech_num_8'] == 0][['total_rech_amt_8','max_rech_amt_8','monthly_2g_8','monthly_3g_8','sachet_2g_8','sachet_3g_8']].nunique()

In [ ]:
telecom_df[telecom_df['total_rech_num_9'] == 0][['total_rech_amt_9','max_rech_amt_9','monthly_2g_9','monthly_3g_9','sachet_2g_9','sachet_3g_9']].nunique()

In [ ]:
print(telecom_df[['date_of_last_rech_6']].isnull().values.any())
print(telecom_df[['date_of_last_rech_7']].isnull().values.any())
print(telecom_df[['date_of_last_rech_8']].isnull().values.any())
print(telecom_df[['date_of_last_rech_9']].isnull().values.any())

In [ ]:
print(telecom_df[['total_rech_num_6']].isnull().values.any())
print(telecom_df[['total_rech_num_7']].isnull().values.any())
print(telecom_df[['total_rech_num_8']].isnull().values.any())
print(telecom_df[['total_rech_num_9']].isnull().values.any())

In [ ]:
total_rech_num = ['total_rech_num_6','total_rech_num_7','total_rech_num_8','total_rech_num_9']
missing_chk = pd.DataFrame(columns= telecom_df.columns)
missing  = pd.DataFrame(columns= telecom_df.columns)
for i in total_rech_num:
    missing = telecom_df[telecom_df[i].isnull()]
    missing_chk = pd.concat([missing,missing_chk])
missing_chk

**Observation** - The column `total_rech_num_6` has no null values. 

This implies that the DataFrame has no meaningful and actual missing values.

**Checking Categorical columns**

In [ ]:
l = []
for i in telecom_df.columns:
    if (telecom_df[i].nunique() <= 5):
        l.append(i)
print(l)

**Dropping the columns that have missing percentage greater than 70%**

In [ ]:
for i in telecom_df.columns:
    if ((telecom_df[i].isnull().sum()/telecom_df.shape[0])*100 > 70):
        telecom_df.drop(i, axis=1,inplace = True)

**Verification** - Verify, whether all the columns whose missing value is should be less than 70

In [ ]:
for i in telecom_df.columns:
    if ((telecom_df[i].isnull().sum()/telecom_df.shape[0])*100 > 70):
        print(i)

**Observation** - All the columns whose missing value greater than 70 has been dropped.

In [ ]:
telecom_df.shape

**Missing value imputation for Numeric columns**

In [ ]:
for i in telecom_df.columns:
    if telecom_df[i].dtype != 'object':
        if ((telecom_df[i].isnull().sum()/telecom_df.shape[0])*100 > 0):
            print(i,":",(telecom_df[i].isnull().sum()/telecom_df.shape[0]) *100)

**Use Simple Imputer from Sklearn to impute `Integer` (datatype) columns with `mean`**

In [ ]:
#Instantiate SimpleImputer
imr = SimpleImputer(missing_values='np.NaN', strategy='mean')

In [ ]:
for i in telecom_df.columns:
    if telecom_df[i].dtype == 'int64':
        if ((telecom_df[i].isnull().sum()/telecom_df.shape[0])*100 > 0):
            imr = imr.fit(telecom_df[[i]])
            telecom_df[i] = imr.transform(telecom_df[[i]])

**Verification** - Verify, whether all the columns with datatype integer has been imputed with `mean`

In [ ]:
for i in telecom_df.columns:
    if telecom_df[i].dtype == 'int64':
        if ((telecom_df[i].isnull().sum()/telecom_df.shape[0])*100 > 0):
            print(i,":",(telecom_df[i].isnull().sum()/telecom_df.shape[0]) *100)
            

**Observation** - All the columns whose datatype is `integer` has been imputed with `mean`(Simple Imputer method).

**Use fillna() to impute `float` (datatype) columns with `mean`**

In [ ]:
for i in telecom_df.columns:
    if telecom_df[i].dtype == 'float64':
        if ((telecom_df[i].isnull().sum()/telecom_df.shape[0])*100 > 0):
            telecom_df[i] = telecom_df[i].fillna(telecom_df[i].mean())

**Verification** - Verify, whether all the columns with datatype float has been imputed with `mean`

In [ ]:
for i in telecom_df.columns:
    if telecom_df[i].dtype == 'float64':
        if ((telecom_df[i].isnull().sum()/telecom_df.shape[0])*100 > 0):
            print(i,":",(telecom_df[i].isnull().sum()/telecom_df.shape[0]) *100)

**Observation** - All the columns whose datatype is `float` has been imputed with `mean`.

**Checking missing values**

We have imputed the missing values for those columns whose datatype is numeric. Now, let us check the missing values for the remaining columns

In [ ]:
X = (telecom_df.isnull().sum()/telecom_df.shape[0])*100
X[X > 0]

***Let us check, how much information can be gathered from the columns last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou***

In [ ]:
telecom_df[['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8']].nunique()

In [ ]:
l = ['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8']
for i in l:
    print(i,":",telecom_df[i].value_counts())

**Observation** - All the above columns have the calendar date. We can drop them.

In [ ]:
telecom_df[['loc_og_t2o_mou','std_og_t2o_mou','loc_ic_t2o_mou']].nunique()

In [ ]:
l = ['loc_og_t2o_mou','std_og_t2o_mou','loc_ic_t2o_mou']
for i in l:
    print(i,":",telecom_df[i].value_counts())

**Observation** - All the above columns have only '0' in it. We can drop them.

In [ ]:
telecom_df.drop(['loc_og_t2o_mou','std_og_t2o_mou','loc_ic_t2o_mou','last_date_of_month_6','last_date_of_month_7','last_date_of_month_8','last_date_of_month_9'],axis=1,inplace = True)

In [ ]:
telecom_df.shape

Instead of dropping the missing values in `date_of_last_rech` columns, it is preferable to impute all these missing date values with a dummy date value **01/01/1900**

In [ ]:
date_cols = ['date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8','date_of_last_rech_9']
for i in date_cols:
    telecom_df[i].fillna('01/01/1900',inplace=True)

In [ ]:
telecom_df[['date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8','date_of_last_rech_9']].isnull().sum()

All the missing values in date columns have been imputed with **01/01/1900**. 

#### Categorical Data 

Fetching the categorical columns from the dataframe `telecom_df`

In [ ]:
categorical_columns = []
for i in telecom_df.columns:
    if telecom_df[i].dtype == 'object':
        categorical_columns.append(i)
print(categorical_columns) 

Checking unique values in each categorical column 

In [ ]:
for i in telecom_df.columns:
    if telecom_df[i].dtype == 'object':
        X = telecom_df[i].nunique()
        print(i,":",X)

Checking for the distribution of category in each column(% of each category under each columns)

In [ ]:
for i in categorical_columns:
    print("======================")
    print(telecom_df[i].value_counts()/len(telecom_df) * 100)
    print()


**Observation** - There are columns whose category is highly skewed with categories whose percentage is less than 2%

Let us replace the highly skewed categories with mode value.

In [ ]:
for i in categorical_columns:
    print("======================")
    # Assigning the value counts to an array
    value_count_array = round(telecom_df[i].value_counts() / len(telecom_df) * 100,2)
    # If the categorical column has more than 1 values
    if len(value_count_array) > 1:
        # Setting a threshold of 1% , the below line will return the boolean array
        threshold = value_count_array <= 2
        # Getting the index of the value_counts less than 2%
        index_of_replacement_values = value_count_array[threshold].index
        # Creating a dictionary which is less than 2% and replacing it with mode value
        a = telecom_df[i].mode()
        replacement_dict = dict.fromkeys(index_of_replacement_values, a[0] )
        # Updating the data frame: Replacing the values in the df which are in the 
        #dictionary replacement_dict with dict value Others
        #ex : telecom_df['date_of_last_rech_6'] = telecom_df['date_of_last_rech_6'].replace('6/20/2014','mode value of the column') 
        telecom_df[i] = telecom_df[i].replace(replacement_dict)
        #print(i)
        print(telecom_df[i].value_counts())
         

In [ ]:
for i in categorical_columns:
    print("======================")
    print(telecom_df[i].value_counts()/len(telecom_df) * 100)
    print()

**Observation** - Highly skewed columns has been replaced with its mode value.                 


The columns **`circle_id`, `mobile_number`** does not give any extra information, it is preferable to drop those columns

In [ ]:
telecom_df.drop(['circle_id','mobile_number'],axis=1,inplace=True)

**OUTLIER TREATMENT**

In [ ]:
numerical_cols = []
for i in telecom_df.columns:
    if(telecom_df[i].dtype == 'int64' or telecom_df[i].dtype == 'float64'):
        numerical_cols.append(i)
print(numerical_cols)

Here, we cannot drop the values, that have high values. There may be a situation we will miss a customer data who is churned. So lets us do **soft capping technique** for outlier treatment.

In [ ]:
for i in numerical_cols:
    q3 = round(telecom_df[i].quantile(0.99),2)
    telecom_df[i] = telecom_df[i].apply(lambda x: q3 if x>=q3 else x) 

In [ ]:
telecom_df.shape

## <span style='color:#0099CC'>Filetering High Value Customers

**High Value Customers:** The `Average_recharge_amt_6_7` (Sum of `total_rech_amt_6` and `total_rech_amt_7`) value is greater than the 70th percentile, those customers will be considered as **High Value Customers**.

In [ ]:
telecom_df['Average_recharge_amt_6_7'] = (telecom_df['total_rech_amt_6']+telecom_df['total_rech_amt_7'])/2
telecom_df['Average_recharge_amt_6_7'].head()

In [ ]:
telecom_df['Average_recharge_amt_6_7'].quantile(0.7)

In [ ]:
telecom_df_high_value = telecom_df[telecom_df['Average_recharge_amt_6_7'] >= telecom_df['Average_recharge_amt_6_7'].quantile(0.7)]
pd.options.display.max_columns = 999
telecom_df_high_value.head()

In [ ]:
telecom_df_high_value[['total_og_mou_9','total_ic_mou_9','vol_2g_mb_9','vol_3g_mb_9']]

**Deriving the CHURN column**

Derive the predictor variable `Churn` based on the value of the columns `total_og_mou_9,total_ic_mou_9,vol_2g_mb_9,vol_3g_mb_9`.

In [ ]:
telecom_df_high_value[(telecom_df_high_value['total_og_mou_9']==0) & (telecom_df_high_value['total_ic_mou_9']==0)
                     & (telecom_df_high_value['vol_2g_mb_9']==0) & (telecom_df_high_value['vol_3g_mb_9']==0)]

All the customers who did not use any of the services in the 9th month must have churned by then. Hence, we will derive the churned customers based on this data

In [ ]:
telecom_df_high_value.insert(loc=len(list(telecom_df_high_value.columns)),column = 'Churn',value=np.zeros(30011).reshape(30011,1))

In [ ]:
telecom_df_high_value['Churn'] = ((telecom_df_high_value['total_og_mou_9']==0) & (telecom_df_high_value['total_ic_mou_9']==0)
                                                             & (telecom_df_high_value['vol_2g_mb_9']==0) & (telecom_df_high_value['vol_3g_mb_9']==0))

In [ ]:
telecom_df_high_value['Churn'] = telecom_df_high_value['Churn'].replace({True:1, False:0})

## <span style='color:#0099CC'>Data Preparation

In [ ]:
telecom_df_high_value.head()

Dropping all the 9th month related columns

In [ ]:
List_of_9 = []

for i in list(telecom_df_high_value.columns):
    if '_9' in i:
        List_of_9.append(i)
    if 'sep' in i:
        List_of_9.append(i)

In [ ]:
telecom_df_high_value.drop(List_of_9,axis=1,inplace=True)

## <span style='color:#0099CC'>Deriving new columns

Aggregating all the **mou** columns to create a single **MOU** column for each month

In [ ]:
List_of_mou6 = []
List_of_mou7 = []
List_of_mou8 = []

for i in list(telecom_df_high_value.columns):
    if '_mou_6' in i:
        List_of_mou6.append(i)
    if '_mou_7' in i:
        List_of_mou7.append(i)
    if '_mou_8' in i:
        List_of_mou8.append(i)
    

In [ ]:
telecom_df_high_value.insert(loc=len(list(telecom_df_high_value.columns)),column = 'Total_MOU_6',value=np.zeros(30011).reshape(30011,1))
telecom_df_high_value.insert(loc=len(list(telecom_df_high_value.columns)),column = 'Total_MOU_7',value=np.zeros(30011).reshape(30011,1))
telecom_df_high_value.insert(loc=len(list(telecom_df_high_value.columns)),column = 'Total_MOU_8',value=np.zeros(30011).reshape(30011,1))

for i in List_of_mou6:
    telecom_df_high_value['Total_MOU_6'] = telecom_df_high_value['Total_MOU_6'] + telecom_df_high_value[i]
    
for i in List_of_mou7:
    telecom_df_high_value['Total_MOU_7'] = telecom_df_high_value['Total_MOU_7'] + telecom_df_high_value[i]
    
for i in List_of_mou8:
    telecom_df_high_value['Total_MOU_8'] = telecom_df_high_value['Total_MOU_8'] + telecom_df_high_value[i]

Dropping the original MOU columns which are now redundant

In [ ]:
telecom_df_high_value.drop(List_of_mou6+List_of_mou7+List_of_mou8, axis=1, inplace=True)

In [ ]:
telecom_df_high_value.head()

In [ ]:
telecom_df_high_value.shape

Aggregating all the **packs** consumed by the customer based on month

In [ ]:
telecom_df_high_value['All_srv_pck_6'] = telecom_df_high_value['monthly_2g_6'] + telecom_df_high_value['sachet_2g_6'] + telecom_df_high_value['monthly_3g_6'] + telecom_df_high_value['sachet_3g_6']
    
telecom_df_high_value['All_srv_pck_7'] = telecom_df_high_value['monthly_2g_7'] + telecom_df_high_value['sachet_2g_7'] + telecom_df_high_value['monthly_3g_7'] + telecom_df_high_value['sachet_3g_7']
    
telecom_df_high_value['All_srv_pck_8'] = telecom_df_high_value['monthly_2g_8'] + telecom_df_high_value['sachet_2g_8'] + telecom_df_high_value['monthly_3g_8'] + telecom_df_high_value['sachet_3g_8']

Dropping the specific service pack columns to reduce redundancy

In [ ]:
telecom_df_high_value.drop(['monthly_2g_6','sachet_2g_6','monthly_3g_6','sachet_3g_6',
                           'monthly_2g_7','sachet_2g_7','monthly_3g_7','sachet_3g_7',
                           'monthly_2g_8','sachet_2g_8','monthly_3g_8','sachet_3g_8'], axis = 1, inplace=True)

In [ ]:
telecom_df_high_value.shape

In [ ]:
telecom_df_high_value.head()

In [ ]:
telecom_df_high_value['Tot_Vol_2/3g_6'] = telecom_df_high_value['vol_2g_mb_6'] + telecom_df_high_value['vol_3g_mb_6'] 

telecom_df_high_value['Tot_Vol_2/3g_7'] = telecom_df_high_value['vol_2g_mb_7'] + telecom_df_high_value['vol_3g_mb_7']

telecom_df_high_value['Tot_Vol_2/3g_8'] = telecom_df_high_value['vol_2g_mb_8'] + telecom_df_high_value['vol_3g_mb_8']

In [ ]:
telecom_df_high_value.drop(['vol_2g_mb_6','vol_3g_mb_6','vol_2g_mb_7','vol_3g_mb_7','vol_2g_mb_8','vol_3g_mb_8'], axis = 1, inplace=True)

In [ ]:
telecom_df_high_value.shape

In [ ]:
telecom_df_high_value.info()

In [ ]:
telecom_df_high_value.head()

As `Date_of_last_rech columns` refer to the last recharge date, we can use this column to find the days since last recharge from a particular date

For easier calculation purpose, let us consider the Oct 1st as a metric and find difference between the dates

In [ ]:
from datetime import date
oct_date = date(2014, 10, 1)
oct_date = pd.to_datetime(oct_date)
telecom_df_high_value["date_of_last_rech_6"]= pd.to_datetime(telecom_df_high_value["date_of_last_rech_6"]) 
telecom_df_high_value["date_of_last_rech_7"]= pd.to_datetime(telecom_df_high_value["date_of_last_rech_7"])
telecom_df_high_value["date_of_last_rech_8"]= pd.to_datetime(telecom_df_high_value["date_of_last_rech_8"])

telecom_df_high_value['Last_rech_days_6'] = (oct_date - telecom_df_high_value['date_of_last_rech_6']).dt.days
telecom_df_high_value['Last_rech_days_7'] = (oct_date - telecom_df_high_value['date_of_last_rech_7']).dt.days
telecom_df_high_value['Last_rech_days_8'] = (oct_date - telecom_df_high_value['date_of_last_rech_8']).dt.days


In [ ]:
telecom_df_high_value[['Last_rech_days_6','Last_rech_days_7','Last_rech_days_8']].head()

Now, we can drop the **Date Last Recharge** columns

Dropping the **date last recharge** columns from the dataset

In [ ]:
telecom_df_high_value.drop(['date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8'],axis=1,inplace=True)

**Scaling the data**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
columns = telecom_df_high_value.columns
telecom_df_high_value = pd.DataFrame(scaler.fit_transform(telecom_df_high_value),columns=columns)

## <span style='color:#0099CC'>Exploratory Data Analysis

***EDA for Total Recharge columns***

EDA for **Total_rech_amt_6** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='total_rech_amt_6')
plt.title('Total Recharge Amount(June) vs Churn')
plt.show()

EDA **Total_rech_amt_7** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='total_rech_amt_7')
plt.title('Total Recharge Amount(July) vs Churn')
plt.show()

**Observation** - As expected, the **total recharge amount** is almost similar in the months June and July as the customers are satisfied with the services

EDA for **Total_rech_amt_8** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='total_rech_amt_8')
plt.title('Total Recharge Amount vs Churn')
plt.show()

**Observation** <br>
From the above box plot, we can infer that 

- Customers who recharged with **higher amount** tend to stay with the network. 
- Customers who recharged with **lower amount** tend to churn from the network. 

The inference can be made that the customers who did not recharge in **August** month tend to churn from the network.

***EDA for MOU Columns***

EDA for **Total_MOU_6** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='Total_MOU_6')
plt.title('Total MOU (June) vs Churn')
plt.show()

EDA for **Total_MOU_7** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='Total_MOU_7')
plt.title('Total MOU (July) vs Churn')
plt.show()

**Observation** <br>
As expected, the customers were satisfied with the services in the month of June and July, hence MOU column is same for both Churners and non-churners

EDA for **Total_MOU_8** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='Total_MOU_8')
plt.title('Total MOU (August) vs Churn')
plt.show()

<br>**Observation**<br>
As expected, the analysis can be summarised as follows:

- Customers whose MOU (Minutes of Usage) is very low, tends to churn 
- Customers whose MOU is high, tend to stay on the network

EDA for **last_day_rch_amt_8** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='last_day_rch_amt_8')
plt.title('Last Day Recharge(August) vs Churn')
plt.show()

In [ ]:
telecom_df_high_value[['last_day_rch_amt_8','Churn']].head()

***EDA `arpu` columns***

EDA for **arpu_6** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='arpu_6')
plt.title('Avg Revenue per user(June) vs Churn')
plt.show()

EDA for **arpu_7** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='arpu_7')
plt.title('Avg Revenue per user(July) vs Churn')
plt.show()

**Observation**
As expected, the average revenue for the customers is same in June and July months for churners and non-churners

EDA for **arpu_8** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='arpu_8')
plt.title('Number of days without recharge(August) vs Churn')
plt.show()

**Observation**
From the box plot, the inference can be summarised as 

- The customers whose average revenue is low, tend to **Churn**
- The customers whose average revenue is high, tend to stay on the network

EDA for **max_rech_amt_8** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='max_rech_amt_8')
plt.title('Max Recharge Amount(August) vs Churn')
plt.show()

**Observation** <br>
The summarization from the box plot is 

- The customers who recharged with **higher amounts**, tend to stay on the network
- The customers who recharged with **lower amounts**, tend to churn from the network

EDA for **All_srv_pck_8** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='All_srv_pck_8')
plt.title('All service packs recharged(August) vs Churn')
plt.show()

**Observation** <br>
From the box plot, it can be referred as 

- People who tend to **Churn**, recharge with very less number of service packs

EDA **total_rech_num_8** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='total_rech_num_8')
plt.title('Total Number of recharges(August) vs Churn')
plt.show()

**Observation** <br>
As expected, customers who recharged with higher amount tend to stay on the network 

But, the customers with lower amount recharge tend to Churn from the network

EDA for **Total_Vol_2/3g_8** column

In [ ]:
sns.boxplot(data=telecom_df_high_value, x='Churn',y='Tot_Vol_2/3g_8')
plt.title('Total Volume consumed(August) vs Churn')
plt.show()

**Observation** <br>
As expected, the customers who stay on the network, tend to consume **large volumes of data**

The customers who Churn from the network, tend to consume **negliglible volume of data**

## <span style='color:#0099CC'>Handling Data Imbalance

The values of `Churn` should be equally disturbed for the better performance of the model.

In [ ]:
telecom_df_high_value['Churn'].value_counts(normalize=True)*100

**Observation:** Here the values of `Churn` is not equally distributed. Let us use **'Smote Technique'** to balance the data.

**Spliting the data into train and test**

In [ ]:
X = telecom_df_high_value.drop('Churn',axis=1)
y = telecom_df_high_value.Churn

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,random_state=100)

## <span style='color:#0099CC'>SMOTE Technique

Implementing **SMOTE** to handle imbalanced data

In [ ]:
sm= SMOTE(sampling_strategy=1.0,random_state=100,k_neighbors=5)
X_train_sampled,y_train_sampled = sm.fit_resample(X_train,y_train)

In [ ]:
print('0 : ', len(y_train_sampled[y_train_sampled == 0]))
print('1 : ', len(y_train_sampled[y_train_sampled == 1]))

Hence, the target variable **Churn** is balanced

## <span style='color:#0099CC'>Model Building - Without PCA

***Implementing Decision Tree Classifier***

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(max_depth=5,min_samples_leaf=10)
dt.fit(X_train_sampled, y_train_sampled)

In [ ]:
# Importing required packages for visualization
from IPython.display import Image  
from six import StringIO
from sklearn.tree import export_graphviz
import pydotplus, graphviz

In [ ]:
dot_data = StringIO()  

export_graphviz(dt, out_file=dot_data, filled=True, rounded=True,
                feature_names=X_train_sampled.columns, 
                class_names=['Not churned', 'Churned'])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

In [ ]:
y_train_pred = dt.predict(X_train_sampled)
y_test_pred = dt.predict(X_test)

In [ ]:
y_train_dt_pred_final = pd.DataFrame({'Actual Churn':y_train_sampled, 'Predicted Churn':y_train_pred})
y_train_dt_pred_final.head()

The accuracy score seems to be good for decision tree. Let us check the other metrics for test data

In [ ]:
y_test_dt_pred_final = pd.DataFrame({'Actual Churn':y_test, 'Predicted Churn':y_test_pred})
y_test_dt_pred_final.head()

In [ ]:
#calculating the accuracy
a = metrics.accuracy_score(y_train_dt_pred_final['Actual Churn'], y_train_dt_pred_final['Predicted Churn'])
print("Train Accuracy:",metrics.accuracy_score(y_train_dt_pred_final['Actual Churn'], y_train_dt_pred_final['Predicted Churn']))

#calculating accuracy for the test data
b = metrics.accuracy_score(y_train_dt_pred_final['Actual Churn'], y_train_dt_pred_final['Predicted Churn'])
print("Test Accuracy:",metrics.accuracy_score(y_train_dt_pred_final['Actual Churn'], y_train_dt_pred_final['Predicted Churn']))

**Observation** - There is not high difference between train and test accuracy scores.

In [ ]:
#Calculating other metrics
confusion_mat = metrics.confusion_matrix(y_test_dt_pred_final['Actual Churn'], y_test_dt_pred_final['Predicted Churn'])
confusion_mat

In [ ]:
TP = confusion_mat[1,1] # true positive 
TN = confusion_mat[0,0] # true negatives
FP = confusion_mat[0,1] # false positives
FN = confusion_mat[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

In [ ]:
data = {'Model Name':['Decision Tree - Without Hyper parameter Tuning'],
        'Training Accuracy':[a],
        'Test Accuracy':[b]
        } 

df = pd.DataFrame(data)
bolded_string = "\033[1m" + "Models and its accuracy" + "\033[0m"
print(' '*20,bolded_string)
      
df

#### Hyper-parameter tuning for the Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
dt = DecisionTreeClassifier(random_state=100)

In [ ]:
params = {
    "max_depth": [2,3,5,10],
    "min_samples_leaf": [20,50,100,500]
}

In [ ]:
grid_search = GridSearchCV(estimator=dt,
                           param_grid=params,
                           cv=5,
                           n_jobs=-1, verbose=1, scoring="accuracy")

In [ ]:
grid_search.fit(X_train_sampled, y_train_sampled)
print(grid_search.best_score_)


In [ ]:
dt_best = grid_search.best_estimator_
dt_best

Fitting the decision tree with best parameters obtained. 

In [ ]:
dt = DecisionTreeClassifier(max_depth=10,min_samples_leaf=20,random_state=100)
dt.fit(X_train_sampled, y_train_sampled)

In [ ]:
y_train_pred = dt.predict(X_train_sampled)
y_test_pred = dt.predict(X_test)

In [ ]:
a = metrics.accuracy_score(y_train_sampled, y_train_pred)
b = metrics.accuracy_score(y_test, y_test_pred)
print("Train Accuracy:",metrics.accuracy_score(y_train_sampled, y_train_pred))
print("Test Accuracy:",metrics.accuracy_score(y_test, y_test_pred))

The accuracy score seems to be a good value for train and test data. 

In [ ]:
#Calculating other metrics for test data
confusion_mat = metrics.confusion_matrix(y_test, y_test_pred)
confusion_mat

In [ ]:
TP = confusion_mat[1,1] # true positive 
TN = confusion_mat[0,0] # true negatives
FP = confusion_mat[0,1] # false positives
FN = confusion_mat[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

The sensitivity reduced compared to the previous models. This implies, this decision tree is failing to predict the **actual churned customers** correctly

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

There is an improvement in specificity, implying this model is predicting the people who did not correctly

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

Let us attempt, hyper parameter tuning with random forest to see if there would be any difference in the accuracy scores

In [ ]:
Results = pd.DataFrame({'Model Name':['Decision Tree - With Hyperparameter Tuning'],
        'Training Accuracy':[a],
        'Test Accuracy':[b]
        }) 

#df = pd.concat([results, tempResults])
df = pd.concat([df,Results])
bolded_string = "\033[1m" + "Models and its accuracy" + "\033[0m"
print(' '*20,bolded_string)
df

### Hyper-parameter tuning for the Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
params = {
    'max_depth': [2,3,5,10],
    'min_samples_leaf': [20,50,100,200],
    'n_estimators': [10, 25, 50]
}



In [ ]:
grid_search = GridSearchCV(estimator=rf,
                           param_grid=params,
                           cv = 4,
                           n_jobs=-1, verbose=1, scoring="accuracy")

In [ ]:
grid_search.fit(X_train_sampled, y_train_sampled)
print(grid_search.best_score_)

In [ ]:
#Identifying best tree
rf_best = grid_search.best_estimator_
rf_best

In [ ]:
y_train_pred = rf_best.predict(X_train_sampled)
y_test_pred = rf_best.predict(X_test)

In [ ]:
a= metrics.accuracy_score(y_train_sampled, y_train_pred)
b = metrics.accuracy_score(y_test, y_test_pred)
print('training accuracy', metrics.accuracy_score(y_train_sampled, y_train_pred))
print('testing accuracy', metrics.accuracy_score(y_test, y_test_pred))

There is an improvement in both training and testing accuracies compared to the decision tree. Let us calculate the other metrics for test data

In [ ]:
#Calculating other metrics for test data
confusion_mat = metrics.confusion_matrix(y_test, y_test_pred)
confusion_mat

In [ ]:
TP = confusion_mat[1,1] # true positive 
TN = confusion_mat[0,0] # true negatives
FP = confusion_mat[0,1] # false positives
FN = confusion_mat[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

The sensitivity improved compared to the previous decision tree model.

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

There is an improvement in specificity, implying this model is predicting the people who did not correctly

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

In [ ]:
Features = pd.DataFrame({
    "Varname": X_train.columns,
    "Important Features": rf_best.feature_importances_
})
Top_10_interpretable_features = Features.sort_values(by="Important Features", ascending=False)
bolded_string = "\033[1m" + "Main Indicators for Churn" + "\033[0m"
print(' '*20,bolded_string)
Top_10_interpretable_features.head(10)

In [ ]:
Results = pd.DataFrame({'Model Name':['Random Forest Classifier - With Hyperparameter Tuning'],
        'Training Accuracy':[a],
        'Test Accuracy':[b]
        }) 

df = pd.concat([df,Results])
bolded_string = "\033[1m" + "Models and its accuracy" + "\033[0m"
print(' '*20,bolded_string)
df

## <span style='color:#0099CC'>Model Building - With PCA (Dimensionality Reduction)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(random_state=100)

In [ ]:
pca.fit(X_train_sampled)

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_ratio_

Plotting the scree plot to determine the number of features required to build models without losing information.

In [ ]:
plt.bar(range(1,len(pca.explained_variance_ratio_)+1), pca.explained_variance_ratio_)

In [ ]:
var_cumu = np.cumsum(pca.explained_variance_ratio_)

Plotting the **Scree Plot**

In [ ]:
var_cumu = np.cumsum(pca.explained_variance_ratio_)
plt.plot(range(1,len(var_cumu)+1), var_cumu)
plt.show()

We can consider **25 principal components** to build the model with almost 

In [ ]:
pc_25 = PCA(n_components=25, random_state=100)

In [ ]:
PC_X_train_transformed = pc_25.fit_transform(X_train_sampled)

In [ ]:
PC_X_test_transformed = pc_25.transform(X_test)

In [ ]:
X_train_transformed = pd.DataFrame(PC_X_train_transformed,columns=['PC'+str(i) for i in range(1,26)])
X_train_transformed.head()

In [ ]:
X_test_transformed = pd.DataFrame(PC_X_test_transformed,columns=['PC'+str(i) for i in range(1,26)])
X_test_transformed.head()

Concatinating the **train and test** dataframes with target variable

In [ ]:
X_train_transformed = pd.concat([X_train_transformed,y_train_sampled],axis=1)
X_train_transformed.head()

In [ ]:
X_test_transformed.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
X_test_transformed = pd.concat([X_test_transformed,y_test],axis=1)
X_test_transformed.head()

In [ ]:
corrmat = np.corrcoef(PC_X_train_transformed.transpose())

In [ ]:
plt.figure(figsize=[15,15])
sns.heatmap(corrmat, annot=True)

As expected the correlation is almost 0 among the features due to PCA

### Logistic Regression Model with PCA

In [ ]:
from sklearn.linear_model import LogisticRegression

Separating the X and y from the data 

In [ ]:
X_train = X_train_transformed.drop('Churn',axis=1)
y_train = X_train_transformed['Churn']

In [ ]:
X_test = X_test_transformed.drop('Churn',axis=1)
y_test = X_test_transformed['Churn']

In [ ]:
#Instantiate logistic regression
log_reg_pca = LogisticRegression()

Logistic_PCA = log_reg_pca.fit(X_train,y_train)

In [ ]:
pred_probability = Logistic_PCA.predict_proba(X_train)
pred_probability[:,1]

Considering only the 2nd column which gives the probabilities of the target variable being 1

In [ ]:
pred_probability[:,1].shape

In [ ]:
y_train_pca_pred = pd.DataFrame({'Actual Churn':y_train, 'Predicted Churn':pred_probability[:,1]})
y_train_pca_pred.head()

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_pca_pred['Actual Churn'], y_train_pca_pred['Predicted Churn'], drop_intermediate = False )

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
draw_roc(y_train_pca_pred['Actual Churn'],y_train_pca_pred['Predicted Churn'])
plt.show()

We will find the optimal cut off

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pca_pred[i]= y_train_pca_pred['Predicted Churn'].map(lambda x: 1 if x > i else 0)
y_train_pca_pred.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pca_pred['Actual Churn'], y_train_pca_pred[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.show()

The optimum seems to be around at 0.5. So, we will implement probability prediction with a threshold **0.5**

In [ ]:
y_train_pca_pred['final_predicted'] = y_train_pca_pred['Predicted Churn'].map( lambda x: 1 if x > 0.5 else 0)

y_train_pca_pred.head()

In [ ]:
y_train_pred = Logistic_PCA.predict(X_train)
y_train_pred[:10]

***Creating a dataframe combining the actual CHURN and predicted CHURN values***

In [ ]:
y_train_pred_final = pd.DataFrame({'Actual Churn':y_train, 'Predicted Churn':y_train_pred})
y_train_pred_final.head()

Calculating the test accuracy

In [ ]:
y_test_pred = Logistic_PCA.predict(X_test)
y_test_pred[:10]

In [ ]:
#calculating the accuracy
y_test_pred_final = pd.DataFrame({'Actual Churn':y_test, 'Predicted Churn':y_test_pred})
y_test_pred_final.head()

In [ ]:
#calculating the Train accuracy
a = metrics.accuracy_score(y_train_pred_final['Actual Churn'], y_train_pred_final['Predicted Churn'])
b = metrics.accuracy_score(y_test_pred_final['Actual Churn'], y_test_pred_final['Predicted Churn'])
print("Train Accuracy:",metrics.accuracy_score(y_train_pred_final['Actual Churn'], y_train_pred_final['Predicted Churn']))
#calculating the Test accuracy
print("Test Accuracy:",metrics.accuracy_score(y_test_pred_final['Actual Churn'], y_test_pred_final['Predicted Churn']))

**Observation** - The Train and Test accuracy is quite good.

In [ ]:
#Calculating other metrics
confusion_mat = metrics.confusion_matrix(y_test_pred_final['Actual Churn'], y_test_pred_final['Predicted Churn'])
confusion_mat

In [ ]:
TP = confusion_mat[1,1] # true positive 
TN = confusion_mat[0,0] # true negatives
FP = confusion_mat[0,1] # false positives
FN = confusion_mat[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

In [ ]:
Results = pd.DataFrame({'Model Name':['Logistic Regression - Using PCA'],
        'Training Accuracy':[a],
        'Test Accuracy':[b]
        }) 

df = pd.concat([df,Results])
bolded_string = "\033[1m" + "Models and its accuracy" + "\033[0m"
print(' '*20,bolded_string)
df

### RandomForestClassifier with PCA

In [ ]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
params = {
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [20,50,100,200],
    'n_estimators': [10, 25, 50]
}

In [ ]:
grid_search = GridSearchCV(estimator=rf,
                           param_grid=params,
                           cv = 10,
                           n_jobs=-1, verbose=1, scoring="accuracy")

In [ ]:
grid_search.fit(X_train, y_train)
print(grid_search.best_score_)

In [ ]:
#Identifying best tree
rf_best = grid_search.best_estimator_
rf_best

In [ ]:
y_train_pred = rf_best.predict(X_train)
y_test_pred = rf_best.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(y_train, y_train_pred)
b = accuracy_score(y_test, y_test_pred)
print("Train_data Accuracy:",accuracy_score(y_train, y_train_pred))
print("Test_data Accuracy:",accuracy_score(y_test, y_test_pred))

In [ ]:
Results = pd.DataFrame({'Model Name':['RandomForestClassifier using PCA'],
        'Training Accuracy':[a],
        'Test Accuracy':[b]
        }) 

df = pd.concat([df,Results])
df

## AdaBoost - With PCA

In [ ]:
shallow_tree = DecisionTreeClassifier(max_depth=2, random_state = 100)
shallow_tree.fit(X_train_transformed, y_train_sampled)

In [ ]:
# adaboost with the tree as base estimator

estimators = list(range(1, 20, 3))

Train_scores = []
Test_scores = []
for n_est in estimators:
    Ada_boost = AdaBoostClassifier(base_estimator=shallow_tree, n_estimators = n_est, random_state=101)
    
    Ada_boost.fit(X_train, y_train)
    y_train_pred = Ada_boost.predict(X_train)
    y_test_pred = Ada_boost.predict(X_test)
    train_score = metrics.accuracy_score(y_train, y_train_pred)
    train_score = round((train_score),2)
    Train_scores.append(train_score)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    test_score = round((test_score),2)
    Test_scores.append(test_score)
a = Train_scores[(len(Train_scores)-1)]
b = Test_scores[(len(Test_scores)-1)]
print("Train_Accuracy:",Train_scores[(len(Train_scores)-1)])
print("Test_Accuracy:",Test_scores[(len(Test_scores)-1)])


In [ ]:
Results = pd.DataFrame({'Model Name':['AdaBoost - With PCA using PCA'],
        'Training Accuracy':[a],
        'Test Accuracy':[b]
        }) 

df = pd.concat([df,Results])
bolded_string = "\033[1m" + "Models and its accuracy" + "\033[0m"
print(' '*20,bolded_string)
df

## XGBoost - With PCA

In [ ]:
xgb_model = xgb.XGBClassifier()

# Default-Run of default-hyperparameters
parameters = {'learning_rate': [0.2,0.3,0.4],
              'max_depth': [3,4,5],
              'min_child_weight': [1,2,3,4,5],
              'n_estimators': [50,75,80]}

scorer = metrics.make_scorer(metrics.roc_auc_score,
                             greater_is_better=True,
                             needs_proba=True,
                             needs_threshold=False)

clf_xgb = model_selection.GridSearchCV(estimator=xgb_model,
                                       param_grid=parameters,
                                       n_jobs=-1,
                                       cv=3,
                                       scoring=scorer,
                                       refit=True)

clf_xgb.fit(X_train, y_train)

In [ ]:
print(clf_xgb.best_params_)
print(clf_xgb.best_score_)
print(clf_xgb.best_estimator_)

In [ ]:
final_model = xgb.XGBClassifier(learning_rate=0.4,
                                max_depth=5,
                                min_child_weight=5,
                                n_estimators=80)

In [ ]:
final_model.fit(X_train, y_train)

In [ ]:
y_xg_train_pred = final_model.predict_proba(X_train)[:, 1]
y_xg_test_pred = final_model.predict_proba(X_test)[:, 1]

In [ ]:
y_xg_train_pred

In [ ]:
y_xg_train_pred.shape

In [ ]:
y_train_xg_pred = pd.DataFrame({'Actual Churn':y_train, 'Predicted Churn':y_xg_train_pred})
y_train_xg_pred.head()

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_xg_pred['Actual Churn'], y_train_xg_pred['Predicted Churn'], drop_intermediate = False )

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
draw_roc(y_train_xg_pred['Actual Churn'],y_train_xg_pred['Predicted Churn'])
plt.show()

The ROC curve is almost ideal. This might be a case of overfitting.

We will find the optimal cut off

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_xg_pred[i]= y_train_xg_pred['Predicted Churn'].map(lambda x: 1 if x > i else 0)
y_train_xg_pred.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_xg_pred['Actual Churn'], y_train_xg_pred[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.show()

The optimum seems to be around at 0.5. So, we will implement probability prediction with a threshold **0.5**

In [ ]:
y_train_xg_pred['final_predicted'] = y_train_xg_pred['Predicted Churn'].map( lambda x: 1 if x > 0.5 else 0)

y_train_xg_pred.head()

In [ ]:
print('Train Score :', metrics.accuracy_score(y_train,y_train_xg_pred['final_predicted']))

The train score is almost 93.9%. Let us check the test score now

In [ ]:
y_xg_test_pred = final_model.predict_proba(X_test)[:,1]

In [ ]:
y_test_xg_pred = pd.DataFrame({'Actual Churn':y_test, 'Predicted Churn':y_xg_test_pred})
y_test_xg_pred.head()

In [ ]:
y_test_xg_pred['final_predicted'] = y_test_xg_pred['Predicted Churn'].map( lambda x: 1 if x > 0.5 else 0)

y_test_xg_pred.head()

In [ ]:
print('Test Score :', metrics.accuracy_score(y_test,y_test_xg_pred['final_predicted']))

In [ ]:
Results = pd.DataFrame({'Model Name':['XGBoost - With PCA'],
        'Training Accuracy':[metrics.accuracy_score(y_train,y_train_xg_pred['final_predicted'])],
        'Test Accuracy':[metrics.accuracy_score(y_test,y_test_xg_pred['final_predicted'])]
        }) 

df = pd.concat([df,Results])
bolded_string = "\033[1m" + "Models and its accuracy" + "\033[0m"
print(' '*20,bolded_string)
df

## <span style='color:#0099CC'>Important Features from interpretatble model - Random Forest Classifier (without PCA)

In [ ]:
Top_10_interpretable_features = Features.sort_values(by="Important Features", ascending=False)
bolded_string = "\033[1m" + "Main Indicators for Churn" + "\033[0m"
print(' '*20,bolded_string)
Top_10_interpretable_features.head(10)

### <span style='color:#599084'>**Inferences** 


From EDA part, it is clearly known that the customers remain satisfied during the month 6(June) and 7(July). They are not satisfied with the serivce during the 8th month(August). As a result, the important features are from the 8th month.

### <span style='color:#599084'>**Recommendations**<br>
The Telecom Industry has to focus on the below features which are the main indicators for Churn.<br>

* total_rech_amt_8 <br>
* Total_MOU_8<br>
* last_day_rch_amt_8<br>
* Last_rech_days_8<br>
* arpu_8<br>
* max_rech_amt_8<br>
* All_srv_pck_8<br>
* total_rech_num_8<br>
* og_others_8<br>
* Tot_Vol_2/3g_8<br>

From the above features, the `Churn` is impacted in the following ways. So, the following features from August month can be used to predict the **Churners beforehand (i.e., 8th month) and provide discounts/offers**. 

* `total_rech_amt_8` column shows that the churners tend to recharge for lesser amounts. 
* `Total_MOU_8` column shows how the customers reduce the usage of the carrier service.
* `arpu_8` column shows that the revenue obtained by the organization through churners' is way less compared to that obtained from non-churners
* `max_rech_amt_8` and `All_srv_pck_8` columns imply that churners either recharged with very less amounts or not recharged at all. 
* `Tot_vol_2/3g_8` columns shows that the volume of 2g/3g pack usage was less by the churners.

### <span style='color:#599084'>**Conclusion** 

* For a smoother and profitable business run, the business can follow the steps:

    - **Churner Identification**<br>
        The above mentioned columns can be used to identify the future churners<br>
        The patterns can be observed in the month based on the above comments
        
    - **Customer Retention**<br>
        Once the future churners are identified, it is suggested to either retain the customers with exciting offers/discounts or get proper feedback from the customers who are predicted to be churned and try to improve the service accordingly.